<a href="https://colab.research.google.com/github/rjsheesh/RJ-Sheesh/blob/master/mag2gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# --- Setup Step: Install aria2 and Mount Google Drive ---
!apt install aria2 -y
!pip install progressbar2
from google.colab import drive
drive.mount('/content/drive')

import subprocess
import os
import time
import progressbar
from concurrent.futures import ThreadPoolExecutor
from google.colab import files

# -- Download Torrent with Magnet link --
def download_magnet(magnet_link, download_path):
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    filename_hint = magnet_link[20:40].replace(':', '').replace('&', '')
    output_file = os.path.join(download_path, filename_hint)

    cmd = [
        "aria2c",
        "--dir", download_path,
        "--seed-time=0",
        "--continue=true",
        "--max-connection-per-server=16",
        "--split=16",
        "--min-split-size=1M",
        "--summary-interval=1",
        "--console-log-level=warn",
        magnet_link
    ]

    try:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            return True
        else:
            return False
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# -- Upload file to Google Drive --
def upload_to_drive(local_path, drive_folder="/content/drive/MyDrive/Torrents/"):
    if not os.path.exists(drive_folder):
        os.makedirs(drive_folder)
    basename = os.path.basename(local_path)
    drive_path = os.path.join(drive_folder, basename)
    subprocess.run(["cp", "-r", local_path, drive_path])
    return drive_path

# -- Download Multiple Magnet Links --
def batch_download_and_upload(magnet_links, download_path="/content/TorrentDownloads"):
    success = []
    fail = []
    uploads = []

    total = len(magnet_links)
    bar = progressbar.ProgressBar(maxval=total, widgets=[
        progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()
    ])
    bar.start()

    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = []
        for idx, magnet in enumerate(magnet_links):
            futures.append(executor.submit(download_magnet, magnet, download_path))

        for idx, future in enumerate(futures):
            result = future.result()
            if result:
                success.append(magnet_links[idx])
            else:
                fail.append(magnet_links[idx])
            bar.update(idx + 1)

    bar.finish()

    print("\n✅ Download শেষ! এখন Drive এ আপলোড করছি...\n")

    # Upload to Google Drive
    for file in os.listdir(download_path):
        local_file_path = os.path.join(download_path, file)
        uploaded_path = upload_to_drive(local_file_path)
        uploads.append(uploaded_path)

    print("\n===== রিপোর্ট =====")
    print(f"✅ Download সফল: {len(success)} টি")
    print(f"❌ Download ব্যর্থ: {len(fail)} টি")
    print(f"✅ Drive এ আপলোড: {len(uploads)} টি ফাইল")

    print("\nআপলোডকৃত ফাইল লিস্ট:")
    for path in uploads:
        print(f"-> {path}")

# -- Main Start --
if __name__ == "__main__":
    print("মামা, একটার পর একটা Magnet Link দাও। শেষে 'DONE' লিখে শেষ করো।\n")

    magnet_list = []
    while True:
        magnet = input("Magnet Link: ").strip()
        if magnet.lower() == "done":
            break
        if magnet.startswith("magnet:?xt="):
            magnet_list.append(magnet)
        else:
            print("❗ ভুল লিঙ্ক, ঠিক করে দাও মামা।")

    if not magnet_list:
        print("কোনো লিঙ্ক পাওয়া যায়নি। বন্ধ করা হলো।")
    else:
        batch_download_and_upload(magnet_list)